In [ ]:
### {“tweet content” :{ Timestamp: DDMMYYYY, Twitter account (str), Num of comments/retweets (int):, Comment sentiment (str):, Tweet sentiment (str):, } }

# 1. request (parameters) --> sqs --> tweepy access data
# 2. (select data) data --> dump to s3
# 3. select data to store in dynamodb?
# 4. conduct sentiment analysis on data --> store in dynamodb? // parallelize by sfn
# 5. spark app?


In [3]:
# item_dict = {
#             'id': survey_data['id'],
#             'TimeStamp': survey_data['created_at'],
#             'Twitter account': survey_data['user']['id_str'],
#             'Num of comments/retweets': survey_data['retweet_count'],
#             'Likes': survey_data['favourites_count'],
#             # 'Comment sentiment': report_times,
#             # 'Tweet sentiment': 
#         }
# item_dict

{'id': 1524707709190352896,
 'TimeStamp': 'Thu May 12 11:07:00 +0000 2022',
 'Twitter account': '26458435',
 'Num of comments/retweets': 11257}

In [7]:
!pip install tweepy

  Using cached tweepy-4.10.0-py3-none-any.whl (94 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.


In [8]:
import tweepy
import json

keys = {'consumer_key' : "tr0R2Csjg2rKqWbaTGnvCOLKo",
        'consumer_secret' : "pqd5aVWeypQq3KrUXxUxY5MTJEwUq25unlDApNRTzq78rUT38K",
        'access_token' : "819043815604375552-gp2URG92uuUTVtNTJbOzbKZMkgG1yey",
        'access_token_secret' : "5FQhsXDoSjlnwqo6ortXWbmqvqRWjpmfFc2ueirAYhn4w"}
auth = tweepy.OAuth1UserHandler(consumer_key=keys.get('consumer_key'), 
                                    consumer_secret=keys.get('consumer_secret'), 
                                    access_token=keys.get('access_token'),
                                    access_token_secret=keys.get('access_token_secret'))
api = tweepy.API(auth)
response = api.search_tweets('abortion', lang = 'en', result_type = 'mixed')
# json.dumps(response, default=str)


In [10]:
len(response)

15

In [12]:
response[0]._json

{'created_at': 'Tue May 31 16:30:00 +0000 2022',
 'id': 1531674365242359808,
 'id_str': '1531674365242359808',
 'text': "It's the filibuster or gun reform. It's the filibuster or abortion. It's the filibuster or voting rights. This shouldn't be a hard decision.",
 'truncated': False,
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'metadata': {'result_type': 'popular', 'iso_language_code': 'en'},
 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 815733290955112448,
  'id_str': '815733290955112448',
  'name': 'Rep. Pramila Jayapal',
  'screen_name': 'RepJayapal',
  'location': 'Seattle, WA',
  'description': 'Congresswoman, lifelong organizer, mom. Proudly serving WA-07. Chair of @USProgressives, Member of @HouseJudiciary, @EdLab

In [1]:
import boto3
import json

sqs = boto3.client('sqs')
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')
dynamodb = boto3.resource('dynamodb')
dynamo_client = boto3.client('dynamodb')
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [2]:
# Create S3 bucket to store raw JSON data
s3.create_bucket(Bucket='raw-tweet-bucket')
s3.create_bucket(Bucket='tweet-text-bucket')

# Create DynamoDB Table for storing processed data
# 'id' will be userID
try:
    table = dynamodb.create_table(
        TableName='tweet_DB',
        KeySchema=[
            {
                'AttributeName': 'id',
                'KeyType': 'HASH'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'id',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1
        }    
    )
except dynamo_client.exceptions.ResourceInUseException:
    table = dynamodb.Table('tweet_DB')

# Wait until AWS confirms that table exists before moving on
table.meta.client.get_waiter('table_exists').wait(TableName='tweet_DB')

In [4]:
# Create Lambda Function
with open('./deployment.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='tweet_lambda',
        Runtime='python3.9',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=3
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip file contents
    response = aws_lambda.update_function_code(
    FunctionName='tweet_lambda',
    ZipFile=lambda_zip
    )

lambda_arn = response['FunctionArn']  

# Create SQS Queue
try:
    queue_url = sqs.create_queue(QueueName='Tweet')['QueueUrl']
except sqs.exceptions.QueueNameExists:
    queue_url = [url
                 for url in sqs.list_queues()['QueueUrls']
                 if 'Tweet' in url][0]
    
sqs_info = sqs.get_queue_attributes(QueueUrl=queue_url,
                                    AttributeNames=['QueueArn'])
sqs_arn = sqs_info['Attributes']['QueueArn']

# Trigger Lambda Function when new data enter SQS Queue
try:
    response = aws_lambda.create_event_source_mapping(
        EventSourceArn=sqs_arn,
        FunctionName='tweet_lambda',
        Enabled=True,
        BatchSize=10
    )
except aws_lambda.exceptions.ResourceConflictException:
    es_id = aws_lambda.list_event_source_mappings(
        EventSourceArn=sqs_arn,
        FunctionName='tweet_lambda'
    )['EventSourceMappings'][0]['UUID']
    
    response = aws_lambda.update_event_source_mapping(
        UUID=es_id,
        FunctionName='tweet_lambda',
        Enabled=True,
        BatchSize=10
    )

# get data about table (should currently be no items in table)
print("SQS -> Lambda Architecture has been Launched")

SQS -> Lambda Architecture has been Launched


In [5]:
def send_request(parameters, sqs_url):
    response = sqs.send_message(QueueUrl=sqs_url,
                                MessageBody=json.dumps(parameters))
                                
    return response['ResponseMetadata']['HTTPStatusCode']
    # return response

In [19]:
param = {
    "keyword": "abortion",
}
send_request(param, sqs_url=queue_url)

200

In [15]:
tweet_batches = [{'batch': []} for i in range(10)]
batch_size = int(len(response)/10)
remaining = len(response)%10
batch_num = 0

for r in response:
    tweet_dict = {
                'id': r._json['id'],
                'TimeStamp': r._json['created_at'],
                'Twitter account': r._json['user']['id_str'],
                'Num of comments/retweets': r._json['retweet_count'],
                'Likes': r._json['favorite_count'],
                'Reply_to': r._json['in_reply_to_user_id']
            }
    tweet_batches[batch_num]['batch'].append(tweet_dict)
    if len(tweet_batches[batch_num]['batch']) == batch_size:
        if remaining > 0:
            remaining -=1 
            continue
    batch_num += 1

tweet_batches

[{'batch': [{'id': 1531674365242359808,
    'TimeStamp': 'Tue May 31 16:30:00 +0000 2022',
    'Twitter account': '815733290955112448',
    'Num of comments/retweets': 5326,
    'Likes': 27356,
    'Reply_to': None},
   {'id': 1531723932767617024,
    'TimeStamp': 'Tue May 31 19:46:58 +0000 2022',
    'Twitter account': '878284831',
    'Num of comments/retweets': 3905,
    'Likes': 13427,
    'Reply_to': None}]},
 {'batch': [{'id': 1531621877726208010,
    'TimeStamp': 'Tue May 31 13:01:26 +0000 2022',
    'Twitter account': '19706851',
    'Num of comments/retweets': 1023,
    'Likes': 2077,
    'Reply_to': None},
   {'id': 1532088007917981696,
    'TimeStamp': 'Wed Jun 01 19:53:40 +0000 2022',
    'Twitter account': '1311079340168404997',
    'Num of comments/retweets': 8,
    'Likes': 0,
    'Reply_to': None}]},
 {'batch': [{'id': 1532088005065945091,
    'TimeStamp': 'Wed Jun 01 19:53:39 +0000 2022',
    'Twitter account': '1224132582717251585',
    'Num of comments/retweets': 532

In [14]:
batch_size

1